##### Copyright 2019 The TensorFlow Authors.

In [34]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Incrustaciones de Palabras (Word Embeddings)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/word_embeddings">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/word_embeddings.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Este tutorial contiene una introducción a las incrustaciones de palabras (word embeddings). Entrenarás tus propias incrustaciones de palabras usando un modelo simple de Keras para una tarea de clasificación de sentimientos, y luego las visualizarás en el [Embedding Projector](http://projector.tensorflow.org).

## Representando texto como números

Los modelos de aprendizaje automático toman vectores (arrays de números) como entrada. Cuando trabajas con texto, lo primero que debes hacer es idear una estrategia para convertir cadenas de texto en números (o "vectorizar" el texto) antes de alimentarlo al modelo. En esta sección, verás tres estrategias para hacerlo.

### Codificaciones One-Hot

Como primera idea, podrías codificar en "one-hot" cada palabra en tu vocabulario. Considera la oración "The cat sat on the mat". El vocabulario (o palabras únicas) en esta oración es (cat, mat, on, sat, the). Para representar cada palabra, crearías un vector de ceros con longitud igual al vocabulario, y luego colocarías un uno en el índice que corresponde a la palabra.

Para crear un vector que contenga la codificación de la oración, podrías concatenar los vectores one-hot de cada palabra.

**Punto clave: Este enfoque es ineficiente.** Un vector codificado en one-hot es disperso (sparse, es decir, la mayoría de los índices son cero). Imagina que tienes 10,000 palabras en el vocabulario. Para codificar en one-hot cada palabra, crearías un vector donde el 99.99% de los elementos son cero.

### Codificar cada palabra con un número único

Un segundo enfoque que podrías probar es codificar cada palabra usando un **número único**. Continuando con el ejemplo anterior, podrías asignar 1 a "cat", 2 a "mat", y así sucesivamente. Luego podrías codificar la oración "The cat sat on the mat" como un vector denso como [5, 1, 4, 3, 5, 2]. Este enfoque es eficiente. En lugar de un vector disperso, ahora tienes uno denso (donde todos los elementos están llenos).

Sin embargo, hay dos desventajas en este enfoque:

* La codificación entera es arbitraria (no captura ninguna relación entre palabras).

* Una codificación entera puede ser desafiante para que un modelo la interprete. Un clasificador lineal, por ejemplo, aprende un solo peso para cada característica. Debido a que no hay relación entre la similitud de dos palabras y la similitud de sus codificaciones, esta combinación característica-peso no es significativa.

### Incrustaciones de palabras (Word embeddings)

Las incrustaciones de palabras nos dan una manera de usar una representación eficiente y densa en la que palabras similares tienen una codificación similar. Es importante destacar que no tienes que especificar esta codificación manualmente. Una incrustación es un vector denso de valores de punto flotante (la longitud del vector es un parámetro que especificas). En lugar de especificar los valores para la incrustación manualmente, son parámetros entrenables (pesos aprendidos por el modelo durante el entrenamiento, de la misma manera que un modelo aprende pesos para una capa densa). Es común ver incrustaciones de palabras de 8 dimensiones (para conjuntos de datos pequeños), hasta 1024 dimensiones cuando se trabaja con conjuntos de datos grandes. Una incrustación de dimensión más alta puede capturar relaciones más sutiles entre palabras, pero requiere más datos para aprender.

## Configuración Inicial

In [ ]:
# ===== IMPORTACIÓN DE LIBRERÍAS NECESARIAS =====
# Estas son todas las bibliotecas que usaremos en este tutorial

import io  # Para operaciones de entrada/salida de archivos
import os  # Para operaciones del sistema operativo (rutas, directorios, etc.)
import re  # Para expresiones regulares (limpieza de texto)
import shutil  # Para operaciones de archivos de alto nivel
import string  # Para constantes de cadenas (puntuación, letras, etc.)
import tensorflow as tf  # Framework principal de deep learning

# Importamos clases específicas de Keras (la API de alto nivel de TensorFlow)
from tensorflow.keras import Sequential  # Para crear modelos secuenciales capa por capa
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D  # Capas del modelo
# Dense: capa totalmente conectada
# Embedding: capa para incrustaciones de palabras
# GlobalAveragePooling1D: capa para promediar secuencias

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization  # Para vectorizar texto

### Descargar el Dataset IMDb
Usarás el [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) a lo largo del tutorial. Entrenarás un modelo clasificador de sentimientos sobre este conjunto de datos y en el proceso aprenderás embeddings desde cero. Para leer más sobre cómo cargar un conjunto de datos desde cero, consulta el [Tutorial de Carga de Texto](../load_data/text.ipynb).

Echa un vistazo al directorio `train/`. Tiene carpetas `pos` y `neg` con reseñas de películas etiquetadas como positivas y negativas respectivamente. Usarás reseñas de las carpetas `pos` y `neg` para entrenar un modelo de clasificación binaria.

In [ ]:
# ===== DEFINIR LA RUTA DEL DATASET =====
# Aquí establecemos la ruta donde se encuentra nuestro conjunto de datos de reseñas de IMDb
# os.getcwd() obtiene el directorio de trabajo actual
# Añadimos "\\data\\" para acceder a la carpeta de datos
dataset_dir = os.getcwd()+ "\\data\\"

A continuación, crea un `tf.data.Dataset` usando `tf.keras.preprocessing.text_dataset_from_directory`. Puedes leer más sobre el uso de esta utilidad en este [tutorial de clasificación de texto](https://www.tensorflow.org/tutorials/keras/text_classification). 

Usa el directorio `train` para crear tanto el conjunto de datos de entrenamiento como el de validación con una división del 20% para validación.

In [ ]:
# ===== CREAR LOS CONJUNTOS DE DATOS DE ENTRENAMIENTO Y VALIDACIÓN =====

# Tamaño del lote: número de ejemplos que se procesan juntos en cada iteración
batch_size = 1024

# Semilla para reproducibilidad: esto asegura que la división de datos sea la misma cada vez
seed = 123

# Crear el conjunto de datos de ENTRENAMIENTO (80% de los datos)
# text_dataset_from_directory lee automáticamente las carpetas y etiqueta los datos
# Las carpetas "pos" y "neg" se usan como etiquetas (1 y 0)
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir,  # Directorio donde están los datos
    batch_size = batch_size,  # Cuántos ejemplos procesar a la vez
    validation_split = 0.2,  # Reservar 20% para validación
    subset = 'training',  # Este es el conjunto de entrenamiento
    seed = seed  # Semilla para reproducibilidad
)

# Crear el conjunto de datos de VALIDACIÓN (20% de los datos)
# La validación se usa para evaluar el modelo durante el entrenamiento
# NO se usa para entrenar, solo para monitorear el rendimiento
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir,
    batch_size = batch_size,
    validation_split = 0.2,  # Mismo 20% que antes
    subset = 'validation',  # Este es el conjunto de validación
    seed = seed  # Misma semilla para consistencia
)

Echa un vistazo a algunas reseñas de películas y sus etiquetas `(1: positivo, 0: negativo)` del conjunto de datos de entrenamiento.

In [ ]:
# ===== EXPLORAR ALGUNOS EJEMPLOS DEL CONJUNTO DE DATOS =====

# Iteramos sobre el conjunto de entrenamiento
# take(1) significa que solo tomamos 1 lote (batch) de datos
for text_batch, label_batch in train_ds.take(1):
  # Dentro de este lote, mostramos solo los primeros 5 ejemplos
  for i in range(5): 
    # Imprimimos la etiqueta (0=negativo, 1=positivo) y el texto de la reseña
    # .numpy() convierte el tensor de TensorFlow a un array de NumPy para mostrarlo
    print(label_batch[i].numpy(), text_batch.numpy()[i])

## Usando la Capa Embedding

Keras facilita el uso de incrustaciones de palabras. Echa un vistazo a la capa [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding).

La capa Embedding puede entenderse como una tabla de búsqueda que mapea desde índices enteros (que representan palabras específicas) a vectores densos (sus incrustaciones). La dimensionalidad (o anchura) de la incrustación es un parámetro con el que puedes experimentar para ver qué funciona bien para tu problema, de la misma manera que experimentarías con el número de neuronas en una capa Dense.

In [ ]:
# ===== CÁLCULO DE LA DIMENSIÓN DE EMBEDDING =====
# Esta es una heurística común: la raíz cuarta del tamaño del vocabulario
# Para un vocabulario de 1000 palabras: 1000^(1/4) ≈ 5.6
# Esto nos da una guía de cuántas dimensiones usar para nuestros embeddings
1000**(1/4)

In [ ]:
# ===== CREAR UNA CAPA DE EMBEDDING DE EJEMPLO =====
# Creamos una capa de embedding con:
# - 1000 palabras posibles en el vocabulario (índices del 0 al 999)
# - 5 dimensiones para cada vector de embedding
# Esto crea una matriz de pesos de tamaño (1000, 5)
# Cada palabra se representará como un vector de 5 números
embedding_layer = tf.keras.layers.Embedding(1000, 5)

Cuando creas una capa Embedding, los pesos para la incrustación se inicializan aleatoriamente (al igual que cualquier otra capa). Durante el entrenamiento, se ajustan gradualmente mediante retropropagación. Una vez entrenados, los embeddings de palabras aprendidos codificarán aproximadamente similitudes entre palabras (tal como fueron aprendidas para el problema específico en el que tu modelo está entrenado).

Si pasas un entero a una capa de embedding, el resultado reemplaza cada entero con el vector de la tabla de embeddings:

In [ ]:
# ===== EJEMPLO DE USO DE LA CAPA EMBEDDING =====
# Pasamos índices de palabras [0, 1, 2, 3, 4, 999] a la capa
# La capa busca cada índice y devuelve su vector de embedding correspondiente
# Por ejemplo, el índice 0 se convierte en un vector de 5 números
# El resultado es una matriz de forma (6, 5): 6 palabras, cada una con 5 dimensiones
result = embedding_layer(tf.constant([0,1,2,3,4,999]))
result.numpy()

In [ ]:
# ===== INSPECCIONAR LOS PESOS DE LA CAPA EMBEDDING =====
# Mostramos la forma de la matriz de pesos (embeddings)
# Debe ser (1000, 5) = 1000 palabras × 5 dimensiones
print(embedding_layer.embeddings.shape)

# Mostramos la matriz completa de embeddings
# Cada fila es el vector de embedding de una palabra
# Estos valores son aleatorios al inicio, pero se entrenan con el modelo
embedding_layer.embeddings

Para problemas de texto o secuencias, la capa Embedding toma un tensor 2D de enteros, de forma `(muestras, longitud_secuencia)`, donde cada entrada es una secuencia de enteros. Puede incrustar secuencias de longitudes variables. Podrías alimentar a la capa de embedding anterior lotes con formas `(32, 10)` (lote de 32 secuencias de longitud 10) o `(64, 15)` (lote de 64 secuencias de longitud 15).

El tensor devuelto tiene un eje más que la entrada, los vectores de embedding se alinean a lo largo del nuevo último eje. Pásale una entrada de lote `(2, 3)` y la salida es `(2, 3, N)`

In [ ]:
# ===== EMBEDDING CON MÚLTIPLES SECUENCIAS =====
# Creamos un tensor 2D con:
# - Primera fila (secuencia): [1, 2, 999]
# - Segunda fila (secuencia): [3, 4, 5]
# Esto representa 2 secuencias de 3 palabras cada una
result = embedding_layer(tf.constant([[1, 2, 999],
                                      [3, 4, 5]]))

# La forma de salida es (2, 3, 5):
# - 2 secuencias
# - 3 palabras por secuencia  
# - 5 dimensiones por embedding de palabra
print(result.shape)

# Mostramos los vectores de embedding resultantes
result.numpy()

Cuando se le da un lote de secuencias como entrada, una capa de embedding devuelve un tensor de punto flotante 3D, de forma `(muestras, longitud_secuencia, dimensionalidad_embedding)`. Para convertir de esta secuencia de longitud variable a una representación fija, hay una variedad de enfoques estándar. Podrías usar una capa RNN, Attention o pooling antes de pasarla a una capa Dense. Este tutorial usa pooling porque es lo más simple. El tutorial [Clasificación de Texto con RNN](text_classification_rnn.ipynb) es un buen próximo paso.

## Preprocesamiento de Texto

A continuación, define los pasos de preprocesamiento del conjunto de datos requeridos para tu modelo de clasificación de sentimientos. Inicializa una capa TextVectorization con los parámetros deseados para vectorizar las reseñas de películas. Puedes aprender más sobre el uso de esta capa en el tutorial de [Clasificación de Texto](https://www.tensorflow.org/tutorials/keras/text_classification).

In [ ]:
# ===== FUNCIÓN DE ESTANDARIZACIÓN PERSONALIZADA =====
# Esta función limpia y normaliza el texto antes de procesarlo

def custom_standardization(input_data):
  """
  Preprocesa el texto de entrada realizando tres pasos:
  1. Convierte todo a minúsculas
  2. Elimina etiquetas HTML '<br />'
  3. Elimina signos de puntuación
  """
  # Paso 1: Convertir todo el texto a minúsculas
  # Esto asegura que "Película" y "película" se traten como la misma palabra
  lowercase = tf.strings.lower(input_data)
  
  # Paso 2: Eliminar las etiquetas de salto de línea HTML '<br />'
  # Las reseñas de IMDb a menudo contienen estas etiquetas
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  
  # Paso 3: Eliminar todos los signos de puntuación (!,.:;? etc.)
  # re.escape convierte caracteres especiales en literales para regex
  # string.punctuation contiene todos los signos de puntuación
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# ===== CONFIGURACIÓN DE PARÁMETROS DE VECTORIZACIÓN =====
# Tamaño del vocabulario: solo las 10,000 palabras más frecuentes
vocab_size = 10000

# Longitud de secuencia: todas las reseñas se cortarán/rellenarán a 100 palabras
sequence_length = 100

# ===== CREAR LA CAPA DE VECTORIZACIÓN =====
# Esta capa convierte texto en secuencias de números
vectorize_layer = TextVectorization(
    standardize = custom_standardization,  # Usar nuestra función de limpieza
    max_tokens = vocab_size,  # Mantener solo las 10,000 palabras más comunes
    output_mode = 'int',  # Salida como enteros (índices de palabras)
    output_sequence_length = sequence_length  # Todas las secuencias tendrán longitud 100
)

# ===== ADAPTAR EL VECTORIZADOR AL CONJUNTO DE DATOS =====
# Crear un dataset solo de texto (sin etiquetas) para construir el vocabulario
# map(lambda x, y: x) extrae solo el texto, descartando las etiquetas (y)
text_ds = train_ds.map(lambda x, y: x)

# adapt() analiza todos los textos y construye el vocabulario
# Cuenta la frecuencia de palabras y crea un diccionario palabra->índice
vectorize_layer.adapt(text_ds)

## Crear un Modelo de Clasificación

Usa la [API Sequential de Keras](https://www.tensorflow.org/guide/keras/sequential_model) para definir el modelo de clasificación de sentimientos. En este caso es un modelo de estilo "Bolsa Continua de Palabras" (Continuous bag of words).

* La capa [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) transforma cadenas de texto en índices de vocabulario. Ya has inicializado `vectorize_layer` como una capa TextVectorization y construido su vocabulario llamando `adapt` en `text_ds`. Ahora vectorize_layer puede usarse como la primera capa de tu modelo de clasificación de extremo a extremo, alimentando cadenas transformadas a la capa Embedding.

* La capa [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) toma el vocabulario codificado en enteros y busca el vector de embedding para cada índice de palabra. Estos vectores se aprenden a medida que el modelo se entrena. Los vectores añaden una dimensión al array de salida. Las dimensiones resultantes son: `(lote, secuencia, embedding)`.

* La capa [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) devuelve un vector de salida de longitud fija para cada ejemplo promediando sobre la dimensión de la secuencia. Esto permite que el modelo maneje entradas de longitud variable, de la manera más simple posible.

* El vector de salida de longitud fija se canaliza a través de una capa totalmente conectada ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) con 16 unidades ocultas.

* La última capa está densamente conectada con un solo nodo de salida.

Precaución: Este modelo no usa enmascaramiento (masking), por lo que el relleno de ceros se usa como parte de la entrada y, por lo tanto, la longitud del relleno puede afectar la salida. Para solucionar esto, consulta la [guía de enmascaramiento y relleno](https://www.tensorflow.org/guide/keras/masking_and_padding).

In [ ]:
# ===== CALCULAR DIMENSIÓN DE EMBEDDING ÓPTIMA =====
# Usando la heurística: raíz cuarta del tamaño del vocabulario
# Para 10,000 palabras: 10000^(1/4) = 10
# Esto nos sugiere usar embeddings de aproximadamente 10-16 dimensiones
10000**(1/4)

In [ ]:
# ===== DEFINIR LA ARQUITECTURA DEL MODELO =====

# Dimensión de los vectores de embedding (cuántos números representan cada palabra)
embedding_dim = 16

# Tamaño del vocabulario (debe coincidir con el vectorize_layer)
vocab_size = 10000

# Crear el modelo secuencial (las capas se apilan una tras otra)
model = Sequential([
    # CAPA 1: Vectorización de texto
    # Convierte texto crudo ("me gusta esta película") en números [45, 234, 12, 567]
    vectorize_layer,
    
    # CAPA 2: Embedding
    # Convierte cada número en un vector de 16 dimensiones
    # Entrada: (batch_size, 100) números enteros
    # Salida: (batch_size, 100, 16) vectores de embeddings
    Embedding(vocab_size, embedding_dim, name='embedding'),
    
    # CAPA 3: Global Average Pooling
    # Promedia todos los embeddings de palabras en una sola representación
    # Entrada: (batch_size, 100, 16)
    # Salida: (batch_size, 16) - un solo vector por reseña
    GlobalAveragePooling1D(),
    
    # CAPA 4: Capa densa oculta con activación ReLU
    # 16 neuronas que aprenden patrones complejos
    # Salida: (batch_size, 16)
    Dense(16, activation='relu'),
    
    # CAPA 5: Capa de salida con activación sigmoide
    # 1 neurona que produce una probabilidad entre 0 y 1
    # 0 = sentimiento negativo, 1 = sentimiento positivo
    # Salida: (batch_size, 1)
    Dense(1, activation='sigmoid')
])

## Compilar y Entrenar el Modelo

Usarás [TensorBoard](https://www.tensorflow.org/tensorboard) para visualizar métricas incluyendo pérdida y precisión. Crea un `tf.keras.callbacks.TensorBoard`.

In [ ]:
# ===== COMPILAR EL MODELO =====
# Configuramos cómo el modelo aprenderá

model.compile(
    # Optimizador: Adam es un algoritmo de optimización adaptativo eficiente
    # Ajusta automáticamente la tasa de aprendizaje durante el entrenamiento
    optimizer='adam',
    
    # Función de pérdida: Binary Crossentropy para clasificación binaria
    # Mide qué tan lejos están las predicciones de las etiquetas reales
    # Valores más bajos = mejores predicciones
    loss = 'binary_crossentropy',
    
    # Métricas: Qué medir durante el entrenamiento
    # Accuracy = porcentaje de predicciones correctas
    metrics = ['accuracy']
)

Compila y entrena el modelo usando el optimizador `Adam` y la pérdida `BinaryCrossentropy`.

In [ ]:
# ===== ENTRENAR EL MODELO =====
# Aquí es donde el aprendizaje realmente ocurre

model.fit(
    # Datos de entrenamiento: el modelo aprende de estos ejemplos
    train_ds,
    
    # Datos de validación: usados para evaluar el modelo después de cada época
    # El modelo NO aprende de estos datos, solo los usa para medir rendimiento
    validation_data = val_ds,
    
    # Épocas: número de veces que el modelo verá TODO el conjunto de entrenamiento
    # En cada época, el modelo pasa por todos los lotes de datos
    # Durante el entrenamiento, verás:
    # - loss: pérdida en entrenamiento (queremos que baje)
    # - accuracy: precisión en entrenamiento (queremos que suba)
    # - val_loss: pérdida en validación (queremos que baje)
    # - val_accuracy: precisión en validación (queremos que suba)
    epochs = 10
)

Con este enfoque, el modelo alcanza una precisión de validación de alrededor del 80% (nota que el modelo está sobreajustando ya que la precisión de entrenamiento es mayor).

Nota: Tus resultados pueden ser un poco diferentes, dependiendo de cómo se inicializaron aleatoriamente los pesos antes de entrenar la capa de embedding.

Puedes ver el resumen del modelo para aprender más sobre cada capa del modelo.

In [ ]:
# ===== MOSTRAR EL RESUMEN DEL MODELO =====
# Esto muestra la arquitectura completa del modelo con:
# - Nombre de cada capa
# - Forma de salida de cada capa
# - Número de parámetros (pesos) en cada capa

# El total de parámetros nos dice cuántos pesos está aprendiendo el modelo
# La mayoría están en la capa Embedding (10,000 palabras × 16 dimensiones = 160,000)
model.summary()

## Recuperar los Embeddings de Palabras Entrenados y Guardarlos en Disco

A continuación, recupera los embeddings de palabras aprendidos durante el entrenamiento. Los embeddings son pesos de la capa Embedding en el modelo. La matriz de pesos tiene forma `(vocab_size, embedding_dimension)`.

Obtén los pesos del modelo usando `get_layer()` y `get_weights()`. La función `get_vocabulary()` proporciona el vocabulario para construir un archivo de metadatos con un token por línea.

In [ ]:
# ===== EXTRAER LOS PESOS DE LA CAPA EMBEDDING =====
# Esta celda solo muestra los pesos, no los guarda aún
# Los pesos son los vectores de embedding que el modelo aprendió
# Cada fila es un vector de 16 números que representa una palabra
model.get_layer('embedding').get_weights()

In [ ]:
# ===== EXTRAER PESOS Y VOCABULARIO =====

# Obtener la matriz de pesos (embeddings) de la capa de embedding
# weights[0] porque get_weights() devuelve una lista, y queremos el primer elemento
# Forma: (10000, 16) - 10,000 palabras, cada una con 16 dimensiones
weights = model.get_layer('embedding').get_weights()[0]

# Obtener la lista de palabras en el vocabulario
# Esto nos da las palabras en orden de su índice
# Por ejemplo: índice 0 = '', índice 1 = '[UNK]', índice 2 = 'the', etc.
vocab = vectorize_layer.get_vocabulary()

In [ ]:
# ===== INSPECCIONAR EL VOCABULARIO =====

# Mostrar el tamaño del vocabulario (debería ser 10,000)
print(len(vocab))

# Mostrar las primeras 10 palabras del vocabulario
# '' = padding (relleno), '[UNK]' = unknown (desconocido)
# Las demás son las palabras más frecuentes
print(vocab[:10])

In [ ]:
# ===== INSPECCIONAR LOS PESOS (EMBEDDINGS) =====

# Mostrar la forma de la matriz de pesos
# Debería ser (10000, 16) = 10,000 palabras × 16 dimensiones
print(weights.shape)

# Mostrar los primeros 2 vectores de embedding
# Estos son los embeddings para el padding ('') y unknown ('[UNK]')
print(weights[:2])

Escribe los pesos en disco. Para usar el [Embedding Projector](http://projector.tensorflow.org), subirás dos archivos en formato separado por tabulaciones: un archivo de vectores (que contiene los embeddings), y un archivo de metadatos (que contiene las palabras).

In [ ]:
# ===== GUARDAR EMBEDDINGS EN ARCHIVOS TSV =====
# Necesitamos 2 archivos para visualizar los embeddings:
# 1. vectors.tsv: contiene los valores numéricos de los embeddings
# 2. metadata.tsv: contiene las palabras correspondientes

# Abrir archivo para escribir los vectores (encoding UTF-8 para caracteres especiales)
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')

# Abrir archivo para escribir los metadatos (las palabras)
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# Iterar sobre cada palabra en el vocabulario
for index, word in enumerate(vocab):
  # Saltar el índice 0 porque es solo padding (relleno), no una palabra real
  if index == 0:
    continue
  
  # Obtener el vector de embedding para esta palabra
  vec = weights[index]
  
  # Escribir el vector en el archivo de vectores
  # Convertir cada número a string y unirlos con tabulaciones
  # Agregar salto de línea al final
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  
  # Escribir la palabra en el archivo de metadatos
  out_m.write(word + "\n")

# Cerrar ambos archivos para asegurar que se guarden correctamente
out_v.close()
out_m.close()

Si estás ejecutando este tutorial en [Colaboratory](https://colab.research.google.com), puedes usar el siguiente fragmento de código para descargar estos archivos a tu máquina local (o usar el navegador de archivos, *View -> Table of contents -> File browser*).

In [ ]:
# ===== DESCARGAR ARCHIVOS EN GOOGLE COLAB (OPCIONAL) =====
# Este código está comentado porque solo funciona en Google Colab
# Si ejecutas este notebook en Colab, descomenta estas líneas para descargar los archivos

# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Visualizar los Embeddings

Para visualizar los embeddings, súbelos al Embedding Projector.

Abre el [Embedding Projector](http://projector.tensorflow.org/) (esto también puede ejecutarse en una instancia local de TensorBoard).

* Haz clic en "Load data" (Cargar datos).

* Sube los dos archivos que creaste arriba: `vectors.tsv` y `metadata.tsv`.

Los embeddings que has entrenado ahora se mostrarán. Puedes buscar palabras para encontrar sus vecinos más cercanos. Por ejemplo, intenta buscar "beautiful" (hermoso). Podrías ver vecinos como "wonderful" (maravilloso).

Nota: Experimentalmente, es posible que puedas producir embeddings más interpretables usando un modelo más simple. Intenta eliminar la capa `Dense(16)`, volver a entrenar el modelo y visualizar los embeddings nuevamente.

Nota: Típicamente, se necesita un conjunto de datos mucho más grande para entrenar embeddings de palabras más interpretables. Este tutorial usa un pequeño conjunto de datos de IMDb con fines de demostración.

## Próximos Pasos

Este tutorial te ha mostrado cómo entrenar y visualizar embeddings de palabras desde cero en un conjunto de datos pequeño.

* Para entrenar embeddings de palabras usando el algoritmo Word2Vec, prueba el tutorial de [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec). 

* Para aprender más sobre procesamiento avanzado de texto, lee el [Modelo Transformer para comprensión del lenguaje](https://www.tensorflow.org/tutorials/text/transformer).